In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_article 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 
AS
WITH 
TEMP AS (
    SELECT * EXCEPT(rn) 
    FROM (
        SELECT 
            a.article product_id,
            a.assortment,
            a.date_from start_date,
            a.date_to end_date,
            a.is_delete,
            CASE WHEN b.product_status LIKE 'Z%' THEN 1 ELSE 0 END block_by_article,
            CASE WHEN c.status = 'ACTIVE' THEN 0 ELSE 1 END block_by_assortment,
            CASE WHEN a.assortment NOT LIKE 'EX_%' THEN 0 ELSE 1 END block_by_ex,
            ROW_NUMBER() OVER (PARTITION BY a.article, a.assortment ORDER BY a.last_update_date DESC, a.proc_date DESC, a.date_to DESC, a.date_from DESC) rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_asm_article a
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.article = b.product_id
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_desc c ON a.assortment = c.assortment
    ) a WHERE rn = 1
)
SELECT * FROM TEMP a
""")